In [1]:
from rl4co.envs import landuseOptEnv
import torch
from rl4co.models import AttentionModel, AttentionModelPolicy
from rl4co.models.nn.env_embeddings.context import LOPContext
from rl4co.models.nn.env_embeddings.dynamic import StaticEmbedding
from rl4co.models.nn.env_embeddings.init import lopInitEmbedding
from rl4co.envs.urbanplan.cityplan import init
import torch
from tensordict.tensordict import TensorDict

D:\Anaconda3\envs\rl4co\lib\site-packages\lightning\fabric\__init__.py:41: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
D:\Anaconda3\envs\rl4co\lib\site-packages\pkg_resources\__init__.py:2317: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
D:\Anaconda3\envs\rl4co\lib\site-packages\lightning\pytorch\__init__.py:37: Deprecated call to `pkg_resources.declare_namespace('lightning.pytorch')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_r

In [2]:
''' neighbour to list '''
polygoncount = 221
neighbourlist = init.getneighbourlist('../Data/queen.csv', polygoncount)
basiclanduse = init.readlanduselist('../Data/baseParcels.shp', polygoncount)
landusePalette = {'Commercial': 'coral',
                  'Residential': 'peachpuff',
                  'Office': 'indianred',
                  'Residential&Commercial': 'lightsalmon',
                  'Green Space': 'lightgreen',
                  'Education': 'lightskyblue',
                  'Hospital': 'royalblue',
                  'SOHO': 'lightcoral'
                  }

landtype = ['Commercial', 'Residential', 'Office', 'Residential&Commercial', 'Green Space', 'Education', 'Hospital',
            'SOHO']
adj_matrix = init.get_adjacency_matrix('../Data/queen.csv', polygoncount)
import math
import numpy as np
shapefile = '../Data/Parcels.shp'


arealist = init.readarealist(shapefile, polygoncount)
locs_list = init.normalizeloc(shapefile)
locs = torch.tensor(locs_list, dtype=torch.float32).unsqueeze(0)
areas = torch.tensor(init.normalizearea(arealist), dtype=torch.float32).unsqueeze(0)
init_plan = torch.tensor(init.map_to_num(basiclanduse, landtype), dtype=torch.int64).unsqueeze(0)
fixed_mask = torch.ones_like(init_plan, dtype=torch.bool)
fixed_mask[(init_plan == 4) | (init_plan == 6)] = 0
neighbourlist = torch.tensor(adj_matrix).unsqueeze(0)
distances = torch.tensor(init.calculate_distance_matrix(locs_list), dtype=torch.float32).unsqueeze(0)
td = TensorDict(
    {
        "locs": locs,
        "areas": areas,
        "init_plan": init_plan,
        "fixed_mask": fixed_mask,
        "adjacency_list": neighbourlist,
        "distances": distances,
    },
    batch_size=1,
)

D:\Anaconda3\envs\rl4co\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [3]:
batch_size = 1024
env = landuseOptEnv(generator_params=dict(num_loc=50))
emb_dim = 128
policy = AttentionModelPolicy(env_name=env.name, # this is actually not needed since we are initializing the embeddings!
                              embed_dim=emb_dim,
                              init_embedding=lopInitEmbedding(emb_dim),
                              context_embedding=LOPContext(emb_dim),
                              dynamic_embedding=StaticEmbedding(emb_dim)
)
# Model: default is AM with REINFORCE and greedy rollout baseline
model = AttentionModel(env,
                       baseline='rollout',
                       policy=policy,
                       )

D:\Anaconda3\envs\rl4co\lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
D:\Anaconda3\envs\rl4co\lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = landuseOptEnv(generator_params=dict(num_loc=50)).to(device)
new_model_checkpoint = AttentionModel.load_from_checkpoint("checkpoints/last.ckpt", strict=False)

D:\Anaconda3\envs\rl4co\lib\site-packages\lightning\pytorch\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['baseline.baseline.policy.encoder.init_embedding.init_embed.weight', 'baseline.baseline.policy.encoder.init_embedding.init_embed.bias', 'baseline.baseline.policy.encoder.net.layers.0.0.module.Wqkv.weight', 'baseline.baseline.policy.encoder.net.layers.0.0.module.Wqkv.bias', 'baseline.baseline.policy.encoder.net.layers.0.0.module.out_proj.weight', 'baseline.baseline.policy.encoder.net.layers.0.0.module.out_proj.bias', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.weight', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.bias', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.running_mean', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.running_var', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.num_batches_tracked', 'baseline.baseline.policy.encoder.net.layers.0.2.module.lins.

reward: 1.001778483390808
compatibility: 7.656000000000001
accessibility: -2.3844751102089616
reward: 0.9676637053489685
compatibility: 7.435500000000002
accessibility: -2.317522324069746
reward: 1.192399501800537
compatibility: 7.642499999999997
accessibility: -2.091311543256259
reward: 0.656539797782898
compatibility: 7.511500000000001
accessibility: -2.824922456238185
reward: 1.1365175247192383
compatibility: 7.488500000000001
accessibility: -2.0926344392396534
reward: 0.7877736687660217
compatibility: 7.431000000000003
accessibility: -2.5849466387006497
reward: 0.7733287215232849
compatibility: 7.412500000000005
accessibility: -2.5967033152843495
reward: 1.1690138578414917
compatibility: 7.471000000000001
accessibility: -2.034514864968714
reward: 0.9809635877609253
compatibility: 7.414999999999997
accessibility: -2.2865903231192357
reward: 1.0408053398132324
compatibility: 7.5675
accessibility: -2.278524154691778
reward: 1.105639100074768
compatibility: 7.527
accessibility: -2.1595

In [ ]:
policy_new = new_model_checkpoint.policy.to(device)
#env = new_model_checkpoint.env.to(device)
td_init = env.reset(batch_size=[1]).to(device)
print(td_init[0])
print(td_init[0].get("locs"))
print(td_init[0].get("areas"))
print(td_init[0].get("init_plan"))
print(td_init[0].get("adjacency_list"))
print(td_init[0].get("distances").tolist())
print(init.map_to_strings(td_init[0].get("init_plan").tolist(), landtype))
out = policy_new(td_init.clone(), env, phase="test", decode_type="greedy", return_actions=True)

# Plotting
print(f"Tour lengths: {[f'{-r.item():.2f}' for r in out['reward']]}")
for td, actions in zip(td_init, out['actions'].cpu()):
    env.render(td, actions)